In [ ]:
import numpy as np
import pandas as pd
import cv2 as cv
from Utils.Preprocessing.MaskCreator import MaskCreator
import json
from tqdm import tqdm
import random

In [ ]:
mask_creator = MaskCreator('files')

In [ ]:
def get_files(patch_size):
    #with open(f'./data/images_{patch_size}.json') as json_file:
    with open(f'D:/Master Thesis/Code/Segmentation/data5/images_{patch_size}_{patch_size // 2}.json') as json_file:
        train_valid_json = json.load(json_file)

    files = []
    for image in train_valid_json['images']:
        for idx, fragment in enumerate(image['fragments']):
            for patch in fragment['patches']:
                files.append({
                    "patch": patch,
                    "name": image['name'],
                    "height": image['height'],
                    "width": image['width'],
                    "fragment": {
                        "idx": idx,
                        "x": fragment['x'],
                        "y": fragment['y'],
                        "width": fragment['width'],
                        "height": fragment['height']
                    }
                })
                
    return files

In [ ]:
files = get_files(512)

In [ ]:
len(files)

In [ ]:
def create_split(mask_creator, patch_size=256, fat_predicted=False, remove_infla=True, remove_endocariums=False):
    df_label = pd.DataFrame()
    files = get_files(patch_size)
    
    annotation_classes = [
        "endocariums"
      ]
    
    for file in tqdm(files, total=len(files)):
        mask_patch = mask_creator.create_mask_from_DB(
                image={
                    "name": file['name'],
                    "height": file['height'],
                    "width": file['width']
                },
                patch=(file['patch'][1], file['patch'][0], patch_size),
                fragment=(file['fragment']['x'], file['fragment']['y'], file['fragment']['width'], file['fragment']['height']),
                annotation_classes=annotation_classes
            )

        labels = {
            'file_name': file['name'],
            'patch_x': file['patch'][1],
            'patch_y': file['patch'][0],
            'height': file['height'],
            'width': file['width'],
            'fragment_idx': file['fragment']['idx'],
            'fragment_x': file['fragment']['x'],
            'fragment_y': file['fragment']['y'],
            'fragment_height': file['fragment']['height'],
            'fragment_width': file['fragment']['width'],
            'type': 'normal'
        }
        
        for idx_c, c in enumerate(annotation_classes):
            unique, count = np.unique(mask_patch[:, :, idx_c], return_counts=True)
            if 1 in unique:
                if len(unique) == 2:
                    labels[c] = count[1]
                    labels['background'] = count[0]
                else: 
                    labels[c] = count[0]
                    labels['background'] = 0
            else:
                labels[c] = 0
                labels['background'] = count[0]

        df_label = pd.concat([
            df_label,
            pd.DataFrame([labels])
        ])
    
    final_df = df_label
    df = df_label
    df = df.sample(frac=1)
    if patch_size == 256:
        valid_df = df[:6000]
        df = df[6000:]
    else:
        valid_df = df[:4500]
        df = df[4500:]
        
    df_label = df

    endocariums_df_vertical = df.copy()
    endocariums_df_horizontal = df.copy()
    endocariums_df_ver_hor = df.copy()

    endocariums_df_vertical['type'] = 'vertical_flip'
    endocariums_df_horizontal['type'] = 'horizontal_flip'

    # Add augmentation for blood vessels
    df = pd.concat([
        df,
        endocariums_df_vertical,
        endocariums_df_horizontal,
        endocariums_df_ver_hor
    ])

    train_filenames = df
    valid_filenames = valid_df

    return final_df, train_filenames, valid_filenames

In [ ]:
def create_jsons(patch_size, step, train_filenames, valid_filenames, test_filenames):
    datasets = [train_filenames, valid_filenames, test_filenames]
    data_types = ['train', 'valid', 'test']
    final_json = {
        "train" : [],
        "valid": [],
        "test": []
    }
    
    for data_type, dataset in zip(data_types, datasets):
        for index, row in dataset.iterrows():
            if row['type'] == 'normal':
                aug = {'type': 'normal'}
            elif row['type'] in ['horizontal_flip', 'horizontalflip']:
                aug = {"type": "flip", "mode": "horizontal"}
            elif row['type'] == 'vertical_flip':   
                aug = {"type": "flip", "mode": "vertical"}
            elif row['type'] == 'horizontal_vertical_flip':
                aug = {'type': 'flip', 'mode': 'horizontal_vertical'}

            final_json[data_type].append({
                'name': row['file_name'],
                'patch': [row['patch_y'], row['patch_x']],
                'height': row['height'],
                'width': row['width'],
                'fragment': {
                    'idx': row['fragment_idx'],
                    'x': row['fragment_x'],
                    'y': row['fragment_y'],
                    'width': row['fragment_width'],
                    'height': row['fragment_height']
                },
                'augmentation': aug
            })
            
    json_string = json.dumps(final_json)
    with open(f"data/srel/images_{patch_size}-{step}-splitted2.json", 'w') as outfile:
        outfile.write(json_string)

In [ ]:
df, train_filenames, valid_filenames = create_split(mask_creator=mask_creator, patch_size=512, fat_predicted=False, remove_infla=False, remove_endocariums=False)

In [ ]:
df.sum()

In [ ]:
train_filenames.sum()

In [ ]:
valid_filenames.sum()

In [ ]:
train_filenames.shape[0]

In [ ]:
valid_filenames.shape[0]

In [ ]:
endocariums_df = df[
    (((df[['blood_vessels', 'endocariums', 'fatty_tissues', 'inflammations']] == 0).sum(axis=1) == 3) & (df['endocariums'] != 0) & (df['type'] == 'normal'))
]

In [ ]:
endocariums_df.sum()

In [ ]:
endocariums_df = endocariums_df.sample(frac=1)[:500]
endocariums_df['type'] = 'horizontal_flip'

test_df = pd.concat([
    df,
    endocariums_df
])

In [ ]:
test_df.sum()

In [ ]:
test_df.shape[0]

In [ ]:
df = test_df

In [ ]:
infla_df = test_df[
    (((test_df[['blood_vessels', 'endocariums', 'fatty_tissues', 'inflammations']] == 0).sum(axis=1) == 3) & (test_df['inflammations'] != 0))
]

In [ ]:
infla_df = infla_df.sample(frac=1)[:500]
test_df = pd.merge(test_df, infla_df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [ ]:
test_df.sum()

In [ ]:
df = test_df

In [ ]:
train_filenames = df

In [ ]:
train_filenames.shape[0]

In [ ]:
train_filenames['type'].unique()

In [ ]:
create_jsons(512, 256, train_filenames, valid_filenames, pd.DataFrame())